In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ocular_Melanomas"
cohort = "GSE60464"

# Input paths
in_trait_dir = "../../input/GEO/Ocular_Melanomas"
in_cohort_dir = "../../input/GEO/Ocular_Melanomas/GSE60464"

# Output paths
out_data_file = "../../output/preprocess/Ocular_Melanomas/GSE60464.csv"
out_gene_data_file = "../../output/preprocess/Ocular_Melanomas/gene_data/GSE60464.csv"
out_clinical_data_file = "../../output/preprocess/Ocular_Melanomas/clinical_data/GSE60464.csv"
json_path = "../../output/preprocess/Ocular_Melanomas/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
import gzip
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on the Series title and summary, this appears to be a gene expression dataset
# The Series_overall_design mentions "expression profiles of a total of 9,829 unique genes"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identifying data rows

# For trait (cerebrotropism):
# Row 2 contains cerebrotropic status with values 0 or 1
trait_row = 2

# For age:
# There is no age information in the sample characteristics
age_row = None

# For gender:
# There is no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> Optional[int]:
    """Convert cerebrotropic status to binary 0/1"""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        parts = value.split(":", 1)
        if len(parts) > 1:
            value = parts[1].strip()
    
    # Extract the actual value after "1=Yes; 0 =No: "
    if "1=Yes; 0 =No:" in value:
        value = value.split("1=Yes; 0 =No:", 1)[1].strip()
    
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age to continuous value"""
    # This function is not needed as age data is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male)"""
    # This function is not needed as gender data is not available
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Read the sample characteristics from the matrix file
    sample_ids = []
    characteristics_data = {}
    
    with gzip.open(os.path.join(in_cohort_dir, "GSE60464_series_matrix.txt.gz"), 'rt') as file:
        for line in file:
            line = line.strip()
            if line.startswith("!Sample_geo_accession"):
                sample_ids = line.split('\t')[1:]
                # Initialize the data structure for each sample
                for sample_id in sample_ids:
                    characteristics_data[sample_id] = {}
            
            # Extract characteristic data
            elif line.startswith("!Sample_characteristics_ch"):
                parts = line.split('\t')
                if len(parts) <= 1:
                    continue
                    
                values = parts[1:]
                
                # Make sure we have the right number of values
                if len(values) != len(sample_ids):
                    continue
                
                # Determine which row this characteristic belongs to
                for i, (sample_id, value) in enumerate(zip(sample_ids, values)):
                    # Store in the dictionary with row numbers as keys
                    characteristics_data[sample_id][int(line.split("ch")[1].split("\t")[0])] = value
    
    # Create DataFrame from the collected data
    clinical_data_dict = {}
    for row_idx in range(10):  # Check rows 0-9
        if any(row_idx in sample_data for sample_data in characteristics_data.values()):
            clinical_data_dict[row_idx] = []
            for sample_id in sample_ids:
                value = characteristics_data[sample_id].get(row_idx, None)
                clinical_data_dict[row_idx].append(value)
    
    # Create a properly formatted DataFrame for geo_select_clinical_features
    clinical_data = pd.DataFrame(clinical_data_dict, index=sample_ids)
    
    # Select and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of processed clinical data:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# We need to first read the clinical data to assess its content
clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_csv(clinical_data_path)
    print(f"Clinical data loaded, shape: {clinical_data.shape}")
    
    # Preview the first few rows to understand the structure
    print("\nPreview of clinical data:")
    sample_char_keys = list(clinical_data.index)
    print(f"Sample characteristics keys: {sample_char_keys}")
    
    # Display unique values for each row to help identify relevant variables
    for idx in clinical_data.index:
        unique_values = clinical_data.loc[idx].unique()
        print(f"Row {idx} unique values: {unique_values}")
else:
    clinical_data = None
    print("Clinical data file not found")

# 1. Gene Expression Data Availability
# Assuming gene expression data is available if a matrix file exists
gene_matrix_path = os.path.join(in_cohort_dir, "gene_expression_matrix.csv")
is_gene_available = os.path.exists(gene_matrix_path)
if is_gene_available:
    print("Gene expression data is available")
else:
    print("Gene expression data is not available")

# 2. Variable Availability and Data Type Conversion
# Based on the output, we'll identify rows for trait, age, and gender

# For demonstration, let's assume:
trait_row = 0  # Example: if ocular melanoma status is in row 0
age_row = None  # Example: if age is not available
gender_row = None  # Example: if gender is not available

def convert_trait(value: str) -> int:
    """
    Convert ocular melanoma trait value to binary format.
    """
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (assuming 1 represents having the condition)
    value = value.lower()
    if 'melanoma' in value or 'positive' in value or 'yes' in value:
        return 1
    elif 'normal' in value or 'negative' in value or 'no' in value or 'control' in value:
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Convert age value to continuous format.
    """
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    try:
        # Extract digits from the value
        import re
        digits = re.findall(r'\d+', value)
        if digits:
            return float(digits[0])
        else:
            return None
    except:
        return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender value to binary format (0 for female, 1 for male).
    """
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    value = value.lower()
    if 'female' in value or 'f' in value:
        return 0
    elif 'male' in value or 'm' in value:
        return 1
    else:
        return None

# 3. Save Metadata for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None and clinical_data is not None:
    print("Extracting clinical features...")
    
    # Extract and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the processed clinical data
    preview = preview_df(selected_clinical_df)
    print("\nPreview of processed clinical data:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Clinical data extraction skipped - trait data not available")


### Step 4: Gene Data Extraction

In [ ]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers in the gene expression data
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols
# These probe IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Examining the gene annotation dictionary to identify relevant columns
# From the preview of gene annotation data, we can see:
# - 'ID' column contains identifiers matching the ones in gene expression data (ILMN_*)
# - 'Symbol' column appears to contain gene symbols

# 2. Create the mapping dataframe linking probe IDs to gene symbols
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')

print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print("\nGene expression data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print("First 10 gene symbols:")
print(gene_data.index[:10].tolist())

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene expression data after normalization:")
print(f"Number of genes: {len(gene_data)}")
print("First 10 gene symbols:")
print(gene_data.index[:10].tolist())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data (already done in step 7)
# We need to load the gene data that was saved in the previous step
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
print(f"Loaded gene expression data with shape: {gene_data.shape}")

# 2. Reload clinical data to properly process it for the Ocular Melanomas dataset
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Let's examine the original data to understand the correct cerebrotropism values
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample Characteristics Dictionary:")
for key, value in sample_characteristics_dict.items():
    print(f"Row {key}: {value}")

# Define conversion function specifically for cerebrotropism status
def convert_trait(value):
    """Convert cerebrotropic status to binary 0/1"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        parts = value.split(":", 1)
        if len(parts) > 1:
            value = parts[1].strip()
    
    # Extract the actual value after "1=Yes; 0 =No: "
    if "1=Yes; 0 =No:" in value:
        value = value.split("1=Yes; 0 =No:", 1)[1].strip()
    
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

# Extract the clinical data using the appropriate row (row 2) based on our analysis
clinical_df = geo_select_clinical_features(
    clinical_data,
    trait=trait,
    trait_row=2,  # Row 2 contains cerebrotropic status (from the sample characteristics dictionary)
    convert_trait=convert_trait,
    # No age or gender data available in this dataset
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

print("Clinical data preview:")
print(preview_df(clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Handle sample ID format to ensure proper linking
# Strip quotes from sample IDs if present
gene_data.columns = gene_data.columns.str.strip('"')
if isinstance(clinical_df.columns, pd.Index):
    clinical_df.columns = clinical_df.columns.str.strip('"')

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains gene expression data from metastatic melanoma samples. The trait variable represents cerebrotropic status (early brain metastasis): 1=early cerebrotropic patients (brain metastases within 6 months), 0=non-cerebrotropic patients (no brain metastases for over 18 months)."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")